In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import sqlalchemy
import boto3
import pyarrow
import pymysql
import mysql.connector
from datetime import datetime, timezone, timedelta
from decouple import config

In [34]:
s3 = boto3.resource(
    service_name ='s3',
    region_name = config('AWS_REGION'),
    aws_access_key_id = config('AWS_KEY'),
    aws_secret_access_key = config('AWS_SECRET')
)

In [33]:
for bucket in s3.buckets.all():
    print(bucket.name)
    

mygamelogsbucket
thebucketofjacob2020


In [36]:
# READ a file directly from s3.  no download.
obj = s3.Bucket('mygamelogsbucket').Object('bby.csv').get()
s3_gamelogs = pd.read_csv(obj['Body'], index_col = 0) # removes the useless index column

In [39]:
### #SQL DATABASE STUFF
mydb = sqlalchemy.create_engine('mysql+mysqlconnector://' + config('AWS_USER') + ':' + config('PW') + '@' + config('IP') + ':' + config('PORT') + '/' + config('DB'),
                     echo = False)


In [10]:
bb1 = pd.read_sql('SELECT * FROM aws_gamelogs;', con = mydb)
bb2 = pd.read_sql_query('SELECT * FROM aws_gamelogs;', con = mydb)


In [ ]:
cnx = mydb.raw_connection()

In [40]:
connection = mysql.connector.connect(host = config('IP'),
                                     port = config('PORT'),
                                      user = config('AWS_USER'),
                                      password = config('PW'),
                                      db = config('DB'))


# connection

all_tables = pd.read_sql_query('SHOW TABLES FROM aws_database;', connection)

In [17]:
df1 = pd.read_sql('SELECT * FROM aws_gamelogs;', con = connection)
df2 = pd.read_sql_query('SELECT * FROM aws_gamelogs;', con = connection)

In [21]:
gamelogs = pd.read_sql_query('SELECT * FROM aws_gamelogs;', connection)

gamelogs_new = gamelogs.groupby(['Team', 'Type', 'GameID']).mean('PTS')

gamelogs_type_aggs = gamelogs[(gamelogs['Type'] != 'Play-In Tournament')] \
    .groupby(['Team', 'Type', 'GameID']) \
    .sum()['PTS'] \
    .groupby(['Team', 'Type']) \
    .mean().round(1) \
    .reset_index() \
    .rename(columns = {'PTS' : 'team_pts'}) \
    .sort_values('team_pts', ascending = False) \
    .assign(language = 'Python7-4')

In [24]:
# gamelogs_type_aggs.to_sql(con = connection, name = "my_python_table", index = False, if_exists = "replace")
# doesnt work, cant use connection var

In [22]:
gamelogs_type_aggs.to_sql(con = mydb, name = "my_python_table", index = False, if_exists = "replace")


In [23]:
new_df = pd.read_sql_query('SELECT * FROM my_python_table;', con = mydb)

In [ ]:
# i can read sql with both mysql connector and sqlalchemy engine
# but can only write to rds with sqlalchemy engine

In [25]:
# top 15 scorers from most recent games
df = pd.read_sql_query('SELECT * FROM aws_gamelogs;', con = mydb)

In [30]:
# start_time = datetime.utcnow()
# my_time = datetime.now().time()
# current_datetime = datetime.today().strftime("%Y-%m-%d %H:%M:%S %p %Z")
# current_date = datetime.now().date()
# print(current_date)
# print(current_datetime)
# print('starting function at', start_time)
yesterday = datetime.now().date() - timedelta(1)

In [55]:
df['Date'] = pd.to_datetime(df['Date']).dt.date

In [57]:
min(df['Date'])
max(df['Date'])

datetime.date(2021, 7, 3)

In [77]:
top10_yesterday = df[df['Date'] == yesterday] \
    .nlargest(10, 'PTS') \
    .sort_values('PTS', ascending = False)
top10_yesterday = df2[['Player', 'Date', 'Outcome', 'Team', 'PTS']]

